In [1]:
from  osgeo import ogr, osr
import psycopg2
import shapely
import shapely.wkt
import geopandas as gpd
%matplotlib inline

In [2]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()

In [3]:
def convertPointToPoly(pointName, polyName,type='Concave',shrink=0.8):
    connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
    cursor = connection.cursor()
    cursor.execute('DROP TABLE IF EXISTS '+polyName)
    cursor.execute('CREATE TABLE '+polyName+' (gid SERIAL PRIMARY KEY, avgscor numeric, numtwts integer, id integer, batchnm integer, geodata character varying(80) COLLATE pg_catalog."default", geom geometry(Polygon), centroid geometry(Point))')
    
    cursor.execute('INSERT INTO '+polyName+' (avgscor, numtwts, geodata, id, batchnm, geom)'+
               '(SELECT d.avgscor, d.numtwts, d.geodata, d.id, d.batchnm, ST_'+type+'HULL(ST_Collect(d.geom), '+str(shrink)+')'+
               ' FROM '+pointName+' AS d'+
               ' GROUP BY (d.id, d.avgscor, d.numtwts, d.geodata, d.batchnm) '+
               ' HAVING ST_GeometryType(ST_'+type+'HULL(ST_Collect(d.geom), '+str(shrink)+')) = \'ST_Polygon\')')
    
    cursor.execute('UPDATE '+polyName+' SET centroid=ST_Centroid(geom)')
    connection.commit()
    
    

In [4]:
driver = ogr.GetDriverByName('ESRI Shapefile')
shp = driver.Open(r'C:\Users\rsarbajna\Documents\Twitter Sentiment Analysis\SQL Change\Twitter Shapefiles\interface_batches\2014-06-01 .shp')

In [5]:
# Get Projection from layer
layer = shp.GetLayer()
spatialRef = layer.GetSpatialRef()
print (spatialRef)

GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4326"]]


In [6]:
# Get Shapefile Fields and Types
layerDefinition = layer.GetLayerDefn()

print ("Name,Type,Width,Precision")
for i in range(layerDefinition.GetFieldCount()):
    fieldName =  layerDefinition.GetFieldDefn(i).GetName()
    fieldTypeCode = layerDefinition.GetFieldDefn(i).GetType()
    fieldType = layerDefinition.GetFieldDefn(i).GetFieldTypeName(fieldTypeCode)
    fieldWidth = layerDefinition.GetFieldDefn(i).GetWidth()
    GetPrecision = layerDefinition.GetFieldDefn(i).GetPrecision()
    print (fieldName + "," + fieldType+ "," + str(fieldWidth) + "," + str(GetPrecision))

Name,Type,Width,Precision
long,Real,24,15
lat,Real,24,15
id,Integer,9,0
dnstyTh,Real,24,15
avgScor,Real,24,15
numTwts,Integer,9,0
stdDev,Real,24,15
batchNm,Integer,9,0
geoData,String,80,0


In [129]:
# cursor.execute("SELECT ST_ConvexHull(ST_Collect(d.the_geom))")
cursor.execute('SELECT * FROM public."2014-06-01 " ORDER BY gid ASC LIMIT 100')
result = cursor.fetchall()

In [130]:
result

[(1,
  Decimal('-78.851667000000006'),
  Decimal('42.493611000000001'),
  1,
  Decimal('0.000300000000000'),
  Decimal('0.888500000000000'),
  1,
  Decimal('0.030024489606718'),
  1,
  'WGS84',
  '010100000047054EB681B653C012F92EA52E3F4540'),
 (2,
  Decimal('-78.895793650000002'),
  Decimal('42.651978759999999'),
  2,
  Decimal('0.000300000000000'),
  Decimal('0.114286666666667'),
  30,
  Decimal('0.030024489606718'),
  1,
  'WGS84',
  '0101000000BAADE3AE54B953C07DF13D0A74534540'),
 (3,
  Decimal('-78.953659999999999'),
  Decimal('42.653143790000001'),
  2,
  Decimal('0.000300000000000'),
  Decimal('0.114286666666667'),
  30,
  Decimal('0.030024489606718'),
  1,
  'WGS84',
  '010100000037E0F3C308BD53C058D138379A534540'),
 (4,
  Decimal('-78.973899000000003'),
  Decimal('42.679000160000001'),
  2,
  Decimal('0.000300000000000'),
  Decimal('0.114286666666667'),
  30,
  Decimal('0.030024489606718'),
  1,
  'WGS84',
  '0101000000DBA6785C54BE53C0E2962C7AE9564540'),
 (5,
  Decimal('-78.93611

In [57]:
cursor.execute('SELECT d.id, ST_AsText(ST_ConvexHull(ST_Collect(d.geom))) FROM public."2014-06-01 " As d GROUP BY d.id;')
result = cursor.fetchall()

In [56]:
cursor.execute('SELECT d.id, ST_GeometryType(ST_ConvexHull(ST_Collect(d.geom))) FROM public."2014-06-01 " As d GROUP BY d.id;')
result = cursor.fetchall()

In [116]:
result;

In [131]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('DROP TABLE IF EXISTS public.june1poly')
cursor.execute('CREATE TABLE public.june1poly (gid SERIAL PRIMARY KEY, avgscor numeric, numtwts integer, id integer, batchnm integer, geodata character varying(80) COLLATE pg_catalog."default", geom geometry(Polygon), centroid geometry(Point))')
connection.commit()

In [132]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('SELECT d.id, d.avgscor, d.numtwts, d.geodata, d.batchnm,'+
               ' ST_AsText(ST_ConvexHull(ST_Collect(d.geom))), ST_GeometryType(ST_ConvexHull(ST_Collect(d.geom)))'+
               ' FROM public."2014-06-01 " As d GROUP BY d.id, d.avgscor, d.numtwts, d.geodata, d.batchnm '+
               ' HAVING ST_GeometryType(ST_ConvexHull(ST_Collect(d.geom))) = \'ST_Polygon\'')
result = cursor.fetchall()

In [108]:
result;

In [133]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('INSERT INTO public.june1poly (avgscor, numtwts, geodata, id, batchnm, geom)'+
               '(SELECT d.avgscor, d.numtwts, d.geodata, d.id, d.batchnm, ST_ConvexHull(ST_Collect(d.geom))'+
               ' FROM public."2014-06-01 " AS d'+
               ' GROUP BY (d.id, d.avgscor, d.numtwts, d.geodata, d.batchnm) '+
               ' HAVING ST_GeometryType(ST_ConvexHull(ST_Collect(d.geom))) = \'ST_Polygon\')')
connection.commit()

In [134]:
cursor.execute("UPDATE public.june1poly SET centroid=ST_Centroid(geom)")
connection.commit()

In [117]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor.execute('SELECT ST_AsText(geom) FROM public.june1poly ORDER BY gid ASC LIMIT 100')
result = cursor.fetchall()
connection.commit()

result;

In [135]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('DROP TABLE IF EXISTS public.june2poly')
cursor.execute('CREATE TABLE public.june2poly (gid SERIAL PRIMARY KEY, avgscor numeric, numtwts integer, id integer, batchnm integer, geodata character varying(80) COLLATE pg_catalog."default", geom geometry(Polygon), centroid geometry(Point))')
connection.commit()

In [136]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('INSERT INTO public.june2poly (avgscor, numtwts, geodata, id, batchnm, geom)'+
               '(SELECT d.avgscor, d.numtwts, d.geodata, d.id, d.batchnm, ST_ConvexHull(ST_Collect(d.geom))'+
               ' FROM public."2014-06-02 " AS d'+
               ' GROUP BY (d.id, d.avgscor, d.numtwts, d.geodata, d.batchnm) '+
               ' HAVING ST_GeometryType(ST_ConvexHull(ST_Collect(d.geom))) = \'ST_Polygon\')')
connection.commit()

In [138]:
cursor.execute("UPDATE public.june2poly SET centroid=ST_Centroid(geom)")
connection.commit()

In [123]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('SELECT * FROM public.june1poly AS j1, public.june2poly AS j2 WHERE j1.id = j2.id GROUP BY j1.gid, j2.gid')
result = cursor.fetchall()
connection.commit()

In [9]:
convertPointToPoly('public."2014-06-03 "', 'public.june3poly', shrink = 0.5)
convertPointToPoly('public."2014-06-04 "', 'public.june4poly', shrink = 0.5)
convertPointToPoly('public."2014-06-05 "', 'public.june5poly', shrink = 0.5)
convertPointToPoly('public."2014-06-06 "', 'public.june6poly', shrink = 0.5)
convertPointToPoly('public."2014-06-07 "', 'public.june7poly', shrink = 0.5)

In [7]:
convertPointToPoly('public."2014-06-07 "', 'public.june7polyconcave', shrink = 0.5)

In [8]:
convertPointToPoly('public."2014-06-07 "', 'public.june7polyconcave75', shrink = 0.75)